# Notebook for the development of module "constraint satisfaction problem for household appliances' scheduling"
### If you have to develop some new function useful also in other situations, please put it in a dedicated .py file, then import it here

### Idea
We can model the constraints and use CSP or even an informed search if we can obtain a good heuristic and test to see which one performs better (in general it should be the heuristic one since we have problem-specific heuristics).
CSP involves discrete variables with finite domain (since every minute can be used).
⇒ tutti i CSP possono essere convertiti in CSP con constraints binari: è un timetabling problem
which class is offered when and where?
HOW TO SOLVE IT:
- Full search
- Backtracking with some improvements on how to choose variables and order, backjumping, no-good, forward checking, constraint propagation (arc consistency)
- Local search: utile se vogliamo fare delle modifiche minime al volo quando cambiano dei requirements \
Anche un SOFT CSP: ogni assignment di valori è associato a un preference value. Di fatto invece che dire “Non posso fare questa cosa” dico “se faccio questa cosa a questa ora è meglio”. Si potrebbe vedere come un Weighted CSP o anche un multi-criteria problem (vogliamo massimizzare/minimizzare due cose assieme)… attenzione che viene fuori anche BRANCH AND BOUND.
Vedi anche CP net


### 1. Real world dataset on machine energy consumption in industry
TODO: understand which one is better:
* https://www.nrel.gov/docs/fy24osti/90442.pdf
* https://ieee-dataport.org/documents/hourly-energy-consumption-industrial-site
* https://github.com/creators01/steel-industry-energy-dataset

### 2. Constraints
Objective function:
* Minimize imported energy
* Minimize cost of imported energy (if this cost changes during the day)

Variables:
* the starting time of every machine. We can suppose that once started it never stops or that it could be paused
* maybe more than one machine for a certain type: this could make things worse

Ideas:
* Time window constraints: some machines may only be allowed to operate during specific times due to labor laws, noise restrictions, or dependencies.
* Mutual exclusion constraints: Some machines cannot run simultaneously due to power load limits or shared resources.
* Max power load per time slot: To avoid peak demand surcharges or circuit overloads, enforce a maximum allowed total consumption per hour. (maybe also the limit could be variable). SOFT CONSTRAINT
* Machine dependencies: Some machines can only start after others finish.
* Minimum/Maximum runtime constraints: Some machines must run at least or at most a certain number of times per day.
* Worker availability: model operator shifts if some machines require manual control.
* The cost of imported energy depends on the hour
* Setup Time or Cooldown Period: Some machines require a startup or cooldown phase between uses.
* Maintenance Windows / Downtime: Certain hours or days are unavailable due to planned maintenance.
* Shared Energy Infrastructure: Some machines might share a line or substation. Their combined use must not exceed the line’s capacity.
* Cyclic operations: Some tasks need to be repeated daily or periodically, not only 3 times a week but at a certain distance
* Priority machines
* Job deadlines: Some processes must finish before a specific time
* Demand Response Signals: If your industry participates in demand response, it may need to reduce load during certain times.

Soft constraints:
* User Preferences or Soft Time Windows

Per la parte 2:
* Battery storage: If the site has batteries, we can model:
  * Charging/discharging rates
  * Battery state of charge
  * Charging only during surplus solar hours

# Model

## Sets
$$
T = \{1, ..., 1440\} = \text{minutes per day}
$$
$$
I = \{1, ...., n\} = \text{machines}
$$
$$
J = \{1, ...., MAX_JOB_N\} = \text{jobs}
$$
## Parameters
$$
 e_{i} = \text{energy used by the machine i per unit of t}
 $$
 $$
 f_{i} = \text{fixed cost for starting the machine i at any time}
 $$
 $$
 p_t = \text{energy produced at time t}
$$
$$
m_t = \text{max energy that we can use at time t for safety reasons}
$$
$$
d_i = \text{duration of the job of machine i}
$$
$$
n_i = \text{number of jobs to complete for machine i}
$$
$$
c_i = \text{time of cooldown for machine i}
$$
## Variables
$$
x_{itj} = \text{if machine i is working at time t for job j}
$$
$$
y_{itj} = \text{if machine i starts working at time t for job j}
$$
$$
z_{t} = \text{value of objective function at time t, real number}
$$
## Model

Here we are minimizing the wasted energy.
Problem: the first constraint doesn't make sense: we can have that the energy we use is lower than the one we produce.
So the objective function should be the max of 0 and the thing that is written there.
$$
\left\{
  \begin{array}{rcr}
    \min \sum_t z_t \\ % be careful: the & symbols are for formatting
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - p_t) \le & z_t & \text{at every time t} \\
         0 \le & z_t & \text{at every time t} \\
         \sum_t \sum_j x_{itj} \ge & n_i d_i \\
         \sum_j x_{itj} \le & 1 \\
         \sum_j y_{itj} \le & 1 \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j}\\
         y_{itj} \le \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} \\
         \sum_{t' \le t} x_{itj} = & d_i & \text{ at some time t and for every job} \\         
  \end{array}
\right.
$$

Model 2:
Try not to use Enel energy. The first constraint tells how much energy would go in battery.
$$
\left\{
  \begin{array}{rcr}
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj}- p_t) \le & 0 & \text{at every time t}\\
         \sum_t \sum_j x_{itj} \ge & n_i d_i \\
         \sum_j x_{itj} \le & 1 \\
         \sum_j y_{itj} \le & 1 \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j}\\
         y_{itj} \le \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} \\
         \sum_{t' \le t} x_{itj} = & d_i & \text{ at some time t and for every job} \\         
  \end{array}
\right.
$$

Model 3: We pay a tariff for every minute that says the maximum energy we take from outside. Call $M_t$ this energy.

$$
\left\{
  \begin{array}{rcr}
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj}- p_t) \ge & M_t & \text{at every time t}\\
         \sum_t \sum_j x_{itj} \ge & n_i d_i \\
         \sum_j x_{itj} \le & 1 \\
         \sum_j y_{itj} \le & 1 \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j}\\
         y_{itj} \le \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} \\
         \sum_{t' \le t} x_{itj} = & d_i & \text{ at some time t and for every job} \\         
  \end{array}
\right.
$$

Model 4: With battery. Every time we do not consume energy, we can re use it after.\
The maximum capacity of a battery is $B$, the number of batteries is $N$, so the total capacity is $NB$. \
After I did model 1, I understand how much capacity I need (it's the value of the objective function). So I put the batteries with that capacity more or less and now I understand if I can survive with only the capacity of the batteries that I just put. Here we are asking if we can survive with only the capacity of the batteries that I just put, but we could also minimize the first constraint and understand how much I can save.

$$
\left\{
  \begin{array}{rcr}
         \sum_t\sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - p_t) \le & 0 \\ % be careful: the & symbols are for formatting
         \sum_{t'<t}\sum_i\sum_j (p_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) \le & NB & \text{at every time t}\\
         \sum_t \sum_j x_{itj} \ge & n_i d_i \\
         \sum_j x_{itj} \le & 1 \\
         \sum_j y_{itj} \le & 1 \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j}\\
         y_{itj} \le \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} \\
         \sum_{t' \le t} x_{itj} = & d_i & \text{ at some time t and for every job} \\         
  \end{array}
\right.
$$

**Constraints:**

0. Every machine must do j_i jobs:
$$
\sum_t x_{it} \ge j_id_i
$$
2. Knapsack contraint that says that not all machines can run at the same time due to max power load for slot (parameter $m_t$, which could depend on time).
   $$
    \sum_i e_{i}x_{it} \le m_t
   $$
3. Some machines may be allowed to run only at specific times: for specific $i$ and for certain $t$. For noise or something like that.
12. Maintenance Windows / Downtime: Certain hours or days are unavailable due to planned maintenance. This should be n° 3
9. Worker availability: model operator shifts if some machines require manual control. This should be n° 3
   $$
   x_{it} = 0
   $$
5. Shared resources: some machines can't run in the same time slot $t$. Like in a pipeline. There are a lot of M', M''.. with overlaps.
   $$
   \sum_{i \in M'} x_{it} \le 1
   $$
6. Once a machine starts, it must go on till it is finished. Or it has a cost to init. Note that in this case at the optimum the machine never gets started twice in a row.
$$            
y_{it} \le x_{it} \le y_{it}+x_{i(t-1)}
$$
7. Machine dependencies: one machine starts only after another has finished.
$$
y_{(k+1)t} \le  \frac{\sum_{t' < t} (x_{kt'})}{d_k}
$$
8. Minimum/Maximum runtime constraints: Some machines must run at least or at most a certain number of times per day. This is done before.
10. The cost of imported energy depends on the hour. TODO AT THE END.
11. Setup Time or Cooldown Period: Some machines require a startup or cooldown phase between uses.
$$
         y_{itj} \le \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i}
$$
13. Shared Energy Infrastructure: Some machines might share a line or substation. Their combined use must not exceed the line’s capacity. TODO: seems a lot number 5.
14. Cyclic operations: Some tasks need to be repeated daily or periodically, not only 3 times a week but at a certain distance. It's like having a cooldown.
15. Priority machines. But this means they should always be on for others to be on. So this doesn't make sense, see the constraint on time to finish a job for priorities.
16. Job deadlines: Some processes must finish before a specific time
    $$
         \sum_{t' \le t} x_{itj} =  d_i  \text{ at some time t and for every job}
    $$

Soft constraints (Soft CSP):
1. User Preferences or Soft Time Windows



## MODEL NUMBER 1
1. Voglio vedere se riesco a fare tutto solo con pannelli solari e batterie. Si, quanto mi costa? (dati 2024)

### Sets
VARIABLES
$$
N = \text{how many batteries to buy}, \,
M = \text{how many panels to buy}, \,
s_t = \text{how much energy we accumulated till now in the batteries}
$$

CONSTANTS
$$
c_b = \text{cost of battery}, \,
c_p = \text{cost of panel}, \,
p_t = \text{energy produced by a panel in a time unit t}
$$

$$
\left\{
  \begin{array}{rcr}
        \min Nc_b+Mc_p \\
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & 0  & \text{for every t}\\
         s_t \le & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j})  & \text{for every t. Should be an equal but in that case it could be more than NB}\\
         0 \le s_t \le & NB & \text{at every time t}\\
         \sum_t x_{itj} \ge & d_i & \text{for every i and j}\\
         \sum_j x_{itj} \le & 1 & \text{for every i and t}\\
         \sum_j y_{itj} \le & 1 & \text{for every i and t}\\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t}\\
         \sum_{t' \le T} x_{it'j} \ge & d_i & \text{ at some time T (threshold) and for every j and i}\\         
  \end{array}
\right.
$$

## PHASE 2
2. Con quello che viene fuori il costo mi chiedo: quanto andrei avanti pagando l'enel e non usando i pannelli con quei soldi? (dati 2024)

Possiamo vedere quanto consumiamo in media e moltiplichiamo

## MODEL 3
3. Se il punto 1 non conviene, cioè esempio ottengo 50 anni dal punto 2,
cerco di minimizzare la mia spesa in questi 50 anni facendo il mix tra energia solare e energia importata dall'esterno (dati 2024)

Costi fissi di comprare cose + energia che consumiamo comprandola da esterno (nei 50 anni di cui sopra)

$$
O = \text{massimo orizzonte temporale}
$$

$$
\left\{
  \begin{array}{rcr}
    \min Nc_b + Mc_p + \sum_{0\le t\le 0} z_t \\
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & z_t & \text{at every time t} \\
         V_t = & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \text{for every t} \\
         V_t \ge & -BIG_M (1 - b_t) & \text{for every t, b\_t = 1 if V\_t > 0} \\
         V_t \le & BIG_M b_t & \text{for every t, b\_t = 0 if V\_t < 0} \\
         s_t \le & V_t + BIG_M (1 - b_t) & \text{for every t} \\
         s_t \le & BIG_M b_t & \text{for every t} \\
         0 \le s_t \le & NB & \text{at every time t}\\
         0 \le & z_t & \text{at every time t} \\
         \sum_t \sum_j x_{itj} \ge & n_i d_i & \text{for every i} \\
         \sum_j x_{itj} \le & 1 & \text{for every i and t} \\
         \sum_j y_{itj} \le & 1 & \text{for every i and t} \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t} \\
         \sum_{t' \le T} x_{itj} = & d_i & \text{ at some time T (threshold) and for every j and i} \\         
  \end{array}
\right.
$$

OLDER ONE, SEE ROW 3 THAT SHOULD BE NOT CORRECT:

$$
\left\{
  \begin{array}{rcr}
    \min Nc_b + Mc_p + \sum_{0\le t\le 0} z_t \\
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & z_t & \text{at every time t} \\
         s_t \le & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \text{for every t. Should be an equal but in that case it could be more than NB. Careful, here it could be negative}\\
         0 \le s_t \le & NB & \text{at every time t}\\
         0 \le & z_t & \text{at every time t} \\
         \sum_t \sum_j x_{itj} \ge & n_i d_i & \text{for every i} \\
         \sum_j x_{itj} \le & 1 & \text{for every i and t} \\
         \sum_j y_{itj} \le & 1 & \text{for every i and t} \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t} \\
         \sum_{t' \le T} x_{itj} = & d_i & \text{ at some time T (threshold) and for every j and i} \\         
  \end{array}
\right.
$$

## MODEL 4
N and M are fixed, it's model 1 or 3 without the variables N and M. Probabilmente abbiamo due modelli diversi a seconda di cosa ci dicono i modelli sopra.

4. Se il punto 1 conviene o il punto 3 mi dice che c'è un buon mix, per avere questo vantaggio faccio ogni giorno lo scheduling per il futuro per non consumare energia (dati giorno per giorno)
I punti 1,2,3 ci fanno capire la strategia migliore. Se è la 1 o la 3 dobbiamo fare un CSP per capire lo scheduling delle macchine giorno per giorno.

Se 1:

$$
\left\{
  \begin{array}{rcr}
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & 0  & \text{for every t}\\
         s_t \le & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j})  & \text{for every t. Should be an equal but in that case it could be more than NB}\\
         0 \le s_t \le & NB & \text{at every time t}\\
         \sum_t \sum_j x_{itj} \ge & n_i d_i & \text{for every i}\\
         \sum_j x_{itj} \le & 1 & \text{for every i and t}\\
         \sum_j y_{itj} \le & 1 & \text{for every i and t}\\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t}\\
         \sum_{t' \le T} x_{it'j} = & d_i & \text{ at some time T (threshold) and for every j and i}\\         
  \end{array}
\right.
$$

Se 3:

$$
\left\{
  \begin{array}{rcr}
    \min \sum_t z_t \\
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & z_t & \text{at every time t} \\
         V_t = & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \text{for every t} \\
         V_t \ge & -BIG_M (1 - b_t) & \text{for every t, b\_t = 1 if V\_t > 0} \\
         V_t \le & BIG_M b_t & \text{for every t, b\_t = 0 if V\_t < 0} \\
         s_t \le & V_t + BIG_M (1 - b_t) & \text{for every t} \\
         s_t \le & BIG_M b_t & \text{for every t} \\
         0 \le s_t \le & NB & \text{at every time t}\\
         0 \le & z_t & \text{at every time t} \\
         \sum_t \sum_j x_{itj} \ge & n_i d_i & \text{for every i} \\
         \sum_j x_{itj} \le & 1 & \text{for every i and t} \\
         \sum_j y_{itj} \le & 1 & \text{for every i and t} \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t} \\
         \sum_{t' \le T} x_{itj} = & d_i & \text{ at some time T (threshold) and for every j and i} \\         
  \end{array}
\right.
$$


### 3. Algorithms to solve the problem
See what we did in class

LOCAL SEARCH:
* we can solve a CSP and then use a local search to optimize. IDEA OF LOCAL SEARCH: start from a feasible point, and then see what are its neighbors. Find the highest value neighbor and see if its value is higher than the one we are using right now. If yes, go to it. (HILL-CLIMBING, problem is I can get stuck in a local minimum/maximum). So we can use:
   * stochastic hill-climbing: choose a neighbor at random from those that are better
   * do a series of hill-climbing and return the best
   * simulated annealing: do a move at random. Accepted if improving, if not then it is accepted with a probability that depends on the time. (REALLY OFTEN USED IN AIRLINE SCHEDULING).
   * LOCAL BEAM: start from K states, at each step go in all neighbors till I find the goal
   * GENETIC ALGORITHMS: combine various states to obtain only one

CSP:
 * to get one value per variable that satisfies all the constraints. The idea is that we use general purpose heuristics rather than problem-specific ones. \
A state is an assignment of values to some or all variables. We want consistent (doesn't violate any constraint) and complete states. \
ANY CSP CAN BE CONVERTED TO A CSP WITH ONLY BINARY CONSTRAINTS.
   * BACKTRACKING SEARCH: start from empty assignment and assign a value that do not conflict (depth first). Backtrack if no legal values.
   * IMPROVED BACKTRACKING:
     * always choose the variable with fewest legal values (break ties choosing the variable involved in the most constraints with unassigned variables).
     * Then, chosen a variable, choose the least constraining value (the one that rules out the fewest values in the other variables).
     * instead of chronological backtracking (backtrack to the previous variable), do BACKJUMPING: backtrack to a variable that might fix the problem.
     * add the constraint on the variables that caused the problem (NO-GOOD) to avoid to redo the error
     * FORWARD CHECKING: if any of the remaining variable has no legal values, terminate.
     * CONSTRAINT PROPAGATION: enforce local consistency. It's a preprocessing step (or also done after assigning a new value). This doesn't guarantee to solve the problem, but helps pruning it:
       * node-consistency: a variable is node-consistent if all the values in the domain satisfy the variable's unary constraints. If I check this, I can delete all the unary constraints
       * arc-consistency (AC-3): a variable is arc-consistent if all the values in the domain satisfy the binary constraints (we are assuming that every other constraint is converted to binary).
       * path-consistency: consistency between more than 2 variables
   * LOCAL SEARCH: start from something that violates the constraints and try to do small changes (one value to one variable) to get the right solution. The search space is all the complete assignments, we need to find one that is consistent. \
To evaluate states we associate the number of constraints it violates.
     * MIN-CONFLICTS: We choose the next value to minimize the number of conflicts.
     * IMPROVED:
       * RESTART: re-initialize the search after tot steps
       * RANDOM WALK: choose something at random but based on probability
       * TABU SEARCH: remember a list of forbidden states (few) to avoid cycling.
       * CONSTRAINT WEIGHTING: eah constraint has a weight. At each step choose a variable with lowest total weight of all violated constraints. (low weight = more important).

LOCAL SEARCH is really useful if I found a good scheduling but there are some minor changes during the week.

CONNECTED COMPONENTS: of the constraint graph are different problems that can be solved alone. For example if it's a tree it's easier. SO WE CAN FIX SOME VARIABLES TO OBTAIN A TREE (TREE DECOMPOSITION).

#### SOFT CONSTRAINTS
Some constraints may have a preference value, like "if I do this it's better". It can also be applied to OR (branch and bound)